In [ ]:
print("Importing libraries...")
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
import numpy as np
import random
from keras.models import load_model
import pickle
import json
print("Loading the Model...")
model = load_model("model.pkl")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
with open('tamil_intents_transports.json') as json_data:
    intents = json.load(json_data)
print("Setting up the model...")
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    bag=np.array(bag)
    return(bag)
ERROR_THRESHOLD = 0.30
def classify(sentence):
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    print(results[0][1]*100)
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def ask(sentence):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))
            results.pop(0)
print("Ask Something with ask() method")
ask('போக்குவரத்து முறைகள் வகைகள் என்ன')